# News

In [1]:
!pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 
!pip install --upgrade google-auth-oauthlib

## Channel Definitions and Setups

In [ ]:
import fetch_videos
CNBC_TV = "UCrp_UI8XtuYfpiqluWLD7Lw"

In [2]:
# Fetch videos
## Getting the videos from the CNBC TV channel
period_type = 'today'  # 'today', 'days', 'weeks', 'months'
number = 2  # The 'today' setting does not use 'number', adjust if using other settings
channel_id = CNBC_TV  # Channel ID as a parameter
start_date, end_date = fetch_videos.get_date_range(period_type, number)
df_videos_today = fetch_videos.fetch_videos(start_date, end_date, channel_id)

In [3]:
# Assume df_videos is your initial DataFrame loaded with video data
hour_range1 = '0-8'
hour_range2 = '8-10'
hour_range3 = '10-12'
hour_range4 = '12-14'
hour_range5 = '14-16'
hour_range6 = '16-24'

# Filter videos from the last 3 days
filtered_df = fetch_videos.filter_videos_by_date_and_time(df_videos_today, 'today', 1)
fetch_videos.display_df(filtered_df)

Title,Published At,Duration (Min),Video ID
European luxury giants send warning signals around consumer spending,2024-07-25 06:12:24-05:00,3,89eoVgBG7UQ
Two strategists discuss where markets go from here following stocks' worst day since 2022,2024-07-25 06:15:45-05:00,6,9sHEhkEZvME
These three stocks could help you AI-proof your portfolio,2024-07-25 06:21:47-05:00,4,hRobVi1s230
Facebook marketing partner on the potential slowdown in digital ads,2024-07-25 06:22:14-05:00,4,zlPyknHbOZo
Emotion-proof your portfolio: Here's what to know,2024-07-25 06:53:58-05:00,3,RKSdJOLBj_4
"Shares of Gucci-owner Kering hit seven-year low after weak forecast, revenue drop on low China sales",2024-07-25 06:54:15-05:00,2,sfVpDtBfQ-k
Southwest profit falls 46% as airline takes ‘urgent’ steps to increase revenue,2024-07-25 06:54:37-05:00,5,K1M-DE3lGCM
"American shares drop 7% on weak profit forecast after backfired sales plan, industry oversupply",2024-07-25 06:54:59-05:00,2,23CAIO-MAjQ
44% of registered voters say they're more motivated to vote after Biden's withdrawal: Betsy App,2024-07-25 06:56:55-05:00,7,R3IHsb4pIew
"Small caps could rise more than 15% in August, says Fundstrat's Tom Lee",2024-07-25 07:06:27-05:00,7,Y4YHF_EYabA


In [4]:
# Add transcripts to the DataFrame
df_videos_with_transcripts = fetch_videos.add_transcripts_to_df(filtered_df)

An error occurred for video ID -mvc3d3tOBY: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=-mvc3d3tOBY! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!


In [6]:
fetch_videos.display_df(df_videos_with_transcripts)

In [7]:
fetch_videos.save_df_to_html(df_videos_with_transcripts, 'videos_with_transcripts.html')

DataFrame has been saved to videos_with_transcripts.html.


## Send emails

In [11]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd
import os

email_user = os.getenv('EMAIL_USER')
email_password = os.getenv('EMAIL_PASSWORD')
email_send1 = 'sliu810@gmail.com'
email_send2 = 'zhengwang827@gmail.com'

recipients = [email_send1]
# Set up the MIME
message = MIMEMultipart()
message['From'] = email_user
message['To'] = email_send1
message['Subject'] = f'CNBC_Videos_{fetch_videos.get_formated_date_today()}'

html_content= fetch_videos.get_html_content(df_videos_with_transcripts)

# Attach the HTML content to the email
message.attach(MIMEText(html_content, 'html'))

# Function to send email
def send_email():
    try:
        server = smtplib.SMTP('smtp-mail.outlook.com', 587)  # Outlook SMTP server
        #server.set_debuglevel(1)  # Enable debugging output
        server.starttls()
        server.login(email_user, email_password)
        text = message.as_string()
        server.sendmail(email_user, recipients, text)
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Send the email
send_email()

Email sent successfully!
